# PySpark – Create DataFrame with Examples
## &copy;  [Omkar Mehta](omehta2@illinois.edu) ##
### Industrial and Enterprise Systems Engineering, The Grainger College of Engineering,  UIUC ###

<hr style="border:2px solid blue"> </hr>

You can manually create a PySpark DataFrame using `toDF()` and `createDataFrame()` methods, both these function takes different signatures in order to create DataFrame from existing RDD, list, and DataFrame.

In [0]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import *

# Dataframe from list
columns = ["language","users_count"]
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

# Create rdd first
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
rdd = spark.sparkContext.parallelize(data)

In [0]:
# Use toDF() from rdd
dfFromRDD1 = rdd.toDF()
dfFromRDD1.printSchema()

root
-- _1: string (nullable = true)
-- _2: string (nullable = true)

In [0]:
# Add columns to it
columns = ["language","users_count"]
dfFromRDD1 = rdd.toDF(columns)
dfFromRDD1.printSchema()

root
-- language: string (nullable = true)
-- users_count: string (nullable = true)

In [0]:
# Use CreateDataFrame on rdd object
dfFromRDD2 = spark.createDataFrame(rdd).toDF(*columns)


In [0]:
# Use CreateDataFrame on list object
dfFromData2 = spark.createDataFrame(data).toDF(*columns)


In [0]:
# Using createDataFrame() with the Row type
rowData = map(lambda x: Row(*x), data) 
dfFromData3 = spark.createDataFrame(rowData,columns)

In [0]:
# Create DataFrame with schema
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)

root
-- firstname: string (nullable = true)
-- middlename: string (nullable = true)
-- lastname: string (nullable = true)
-- id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
firstname|middlename|lastname|id |gender|salary|
+---------+----------+--------+-----+------+------+
James | |Smith |36636|M |3000 |
Michael |Rose | |40288|M |4000 |
Robert | |Williams|42114|M |4000 |
Maria |Anne |Jones |39192|F |4000 |
Jen |Mary |Brown | |F |-1 |
+---------+----------+--------+-----+------+------+

In [0]:
# Create DataFrame from Data sources
df2 = spark.read.csv("/FileStore/tables/covid_analytics_clinical_data.csv")
#df2.show()

In [0]:
# Creating from text (TXT) file
df2 = spark.read.text("/FileStore/tables/data.txt")


In [0]:
# Creating from JSON file
df2 = spark.read.json("/FileStore/tables/example_1.json")


# Spark Check Column Present in DataFrame

You can get all columns of a DataFrame as an Array[String] by using columns attribute of Spark DataFrame and use this with Scala Array functions to check if a column/field present in DataFrame, In this article I will also cover how to check if a column present/exists in nested column and by case insensitive.

### Check Column Exists by Case insensitive

In [0]:
%scala 
df2 = spark.read.csv("/FileStore/tables/covid_analytics_clinical_data.csv")


df2.columns.map(_.toUpperCase).contains(columnNameToCheck.toUpperCase)


# PySpark Read and Write Parquet File

In [0]:
df.write.parquet("/tmp/out/people.parquet") 
parDF1=spark.read.parquet("/temp/out/people.parquet")

In [0]:

data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)

In [0]:
# Pyspark Write DataFrame to Parquet file format
df.write.parquet("/FileStore/tables/people.parquet")


In [0]:
# Pyspark Read Parquet file into DataFrame
parDF=spark.read.parquet("/FileStore/tables/people.parquet")

In [0]:
# # Append or Overwrite an existing Parquet file
# df.write.mode('append').parquet("/tmp/output/people.parquet")
# df.write.mode('overwrite').parquet("/tmp/output/people.parquet")

In [0]:
# Executing SQL queries DataFrame
parDF.createOrReplaceTempView("ParquetTable")
parkSQL = spark.sql("select * from ParquetTable where salary >= 4000 ")


In [0]:
# Creating a table on Parquet file
spark.sql("CREATE TEMPORARY VIEW PERSON USING parquet OPTIONS (path \"/FileStore/tables/people.parquet\")")
spark.sql("SELECT * FROM PERSON").show()

+---------+----------+--------+-----+------+------+
firstname|middlename|lastname| dob|gender|salary|
+---------+----------+--------+-----+------+------+
 Robert | |Williams|42114| M| 4000|
 Michael | Rose| |40288| M| 4000|
 James | | Smith|36636| M| 3000|
 Maria | Anne| Jones|39192| F| 4000|
 Jen| Mary| Brown| | F| -1|
+---------+----------+--------+-----+------+------+

In [0]:
# Create Parquet partition file

df.write.partitionBy("gender","salary").mode("overwrite").parquet("/FileStore/tables/people2.parquet")


In [0]:
# Retrieving from a partitioned Parquet file

parDF2=spark.read.parquet("/FileStore/tables/people2.parquet/gender=M")
parDF2.show(truncate=False)

+---------+----------+--------+-----+------+
firstname|middlename|lastname|dob |salary|
+---------+----------+--------+-----+------+
Michael |Rose | |40288|4000 |
James | |Smith |36636|3000 |
Robert | |Williams|42114|4000 |
+---------+----------+--------+-----+------+

In [0]:
# Creating a table on Partitioned Parquet file

spark.sql("CREATE TEMPORARY VIEW PERSON2 USING parquet OPTIONS (path \"/FileStore/tables/people2.parquet/gender=F\")")
spark.sql("SELECT * FROM PERSON2" ).show()

+---------+----------+--------+-----+------+
firstname|middlename|lastname| dob|salary|
+---------+----------+--------+-----+------+
 Jen| Mary| Brown| | -1|
 Maria | Anne| Jones|39192| 4000|
+---------+----------+--------+-----+------+